In [1]:
import plotly.figure_factory as ff

import numpy as np
import pandas as pd
import os

os.chdir(r'C:\Users\borde\Desktop\COVID\Risk Locator')
Rates_pd = pd.read_excel('Output.xlsx',index_col=False)
#Rates_pd = Rates_pd.drop(['index'], axis=1)
Rates_pd = Rates_pd[Rates_pd['FIPS']>1000]
Rates_pd = Rates_pd[Rates_pd['FIPS']<60000].reset_index()
TrackedFIPS_pd = pd.DataFrame(list(zip(Rates_pd['FIPS'], Rates_pd['Incidence_Rate'])), columns = ['FIPS', 'Incidence_Rate'])

os.chdir(r'C:\Users\borde\Desktop\COVID')
FIPS_pd = pd.read_csv('FIPS Lookup.csv',index_col=False)
FIPS_pd = FIPS_pd.drop(['UID', 'iso2', 'iso3', 'code3', 'Country_Region','Lat', 'Long_', 'Combined_Key', 'Population'],axis=1).dropna(subset=['FIPS'])
FIPS_pd = FIPS_pd[FIPS_pd['FIPS']>1000]
FIPS_pd = FIPS_pd[FIPS_pd['FIPS']<60000].reset_index()
AllFIPS_pd = pd.DataFrame(list(zip(FIPS_pd['FIPS'])), columns = ['FIPS'])

missing_pd = pd.merge(AllFIPS_pd, TrackedFIPS_pd, how='outer', indicator=True)

std = Rates_pd['Incidence_Rate'].std()
mean = Rates_pd['Incidence_Rate'].mean()

missing_pd['Incidence_Rate'] = missing_pd['Incidence_Rate'].fillna(0)

text = []

for x in range(0,len(missing_pd)):
    if missing_pd['Incidence_Rate'][x]<=mean + std:
        text.append('Incidence Rate: ' + str(missing_pd['Incidence_Rate'][x]) + '\nRisk: LOW')
    elif missing_pd['Incidence_Rate'][x]>mean + std and missing_pd['Incidence_Rate'][x]<=mean+2*std:
        text.append('Incidence Rate: ' + str(missing_pd['Incidence_Rate'][x]) + '\nRisk: MEDIUM')
    elif missing_pd['Incidence_Rate'][x]>mean+2*std:
        text.append('Incidence Rate: ' + str(missing_pd['Incidence_Rate'][x]) + '\nRisk: HIGH')
    else: 
        text.append('ERROR')

missing_pd['Text'] = text

Alaska_pd = missing_pd[missing_pd['FIPS']<3000]
Alaska_pd = Alaska_pd[Alaska_pd['FIPS']>2000].reset_index()

Hawaii_pd = missing_pd[missing_pd['FIPS']<16000]
Hawaii_pd = Hawaii_pd[Hawaii_pd['FIPS']>15000].reset_index()

#display(missing_pd[missing_pd['_merge']!='both'])

#colorscale = ["#CCFFCC","#99FF99","#66FF66","#00FF00","#99FF00","#CCFF00","#FFFF00","#FFCC00","#FF6600","#FF0000"]
colorscale = ["#F1D2D6","#F4C2CB","#F2AAB7","#E98598","#D4596F","#BC394B","#9F292C", "#7A2F3D"]

endpts = list(np.linspace(1, mean+2*std, len(colorscale) - 1))
fips = missing_pd['FIPS'].tolist()
values = missing_pd['Incidence_Rate'].tolist()

fig = ff.create_choropleth(
    fips=fips, values=values,
    binning_endpoints=endpts,
    colorscale=colorscale,
    show_state_data=True, 
    show_hover=True, centroid_marker={'opacity': 0},
    asp=2.9,
    plot_bgcolor='#000033', paper_bgcolor='#000033',state_outline={'color': "#333333", 'width':0.5}
)

fig2 = ff.create_choropleth(
    fips=Alaska_pd['FIPS'], values=Alaska_pd['Incidence_Rate'],
    binning_endpoints=endpts,
    colorscale=colorscale,scope=['Alaska'],
    show_state_data=True,
    show_hover=True, centroid_marker={'opacity': 0},
    asp=2.9,
    plot_bgcolor='#000033', paper_bgcolor='#000033'
)

fig3 = ff.create_choropleth(
    fips=Hawaii_pd['FIPS'], values=Hawaii_pd['Incidence_Rate'],
    binning_endpoints=endpts,
    colorscale=colorscale,scope=['Hawaii'],
    show_state_data=False,
    show_hover=True, centroid_marker={'opacity': 0},
    asp=2.9,
    plot_bgcolor='#000033', paper_bgcolor='#000033'
)


fig.layout.template = None
fig2.layout.template = None
fig3.layout.template = None
fig.update_layout(margin=dict(l=120, r=0, b=95, t=60), showlegend=False)
fig.update_layout(font_color="white",legend=dict(yanchor='bottom', y = .05, xanchor='right', x = .95, font=dict(color="white")))
fig2.update_layout(margin=dict(l=20, r=0, b=0, t=0),showlegend=False)
fig3.update_layout(margin=dict(l=0, r=0, b=20, t=0),showlegend=False)

print('working..')
fig.write_html('first_figure.html',auto_open=True)
fig2.write_html('second_figure.html',auto_open=True)
fig3.write_html('third_figure.html',auto_open=True)
print('done')

working..
done
